In [93]:
import pandas as pd

In [94]:
df = pd.read_excel('data_jobs_salary_all.xlsx')

In [95]:
df.head()

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills
0,Senior Data Scientist,Senior Data Scientist Data and Analytics Perfo...,"Bennington, NE",via ZipRecruiter,Full-time,False,Sudan,2023-04-24 09:51:15,False,True,Sudan,year,128050.0,NaN,Cox Communications,"['sql', 'python', 'aws', 'pyspark', 'tableau',..."
1,Data Engineer,Data Engineer - MA,"Mesa, AZ",via Indeed,Full-time,False,Georgia,2023-03-13 12:51:23,True,True,United States,year,140000.0,NaN,Worldgate LLC,"['sql', 'nosql', 'java', 'python', 'kafka', 's..."
2,Senior Data Analyst,Supervisory Information Technology Specialist ...,"Alexandria, VA",via ZipRecruiter,Full-time,False,"New York, United States",2023-07-05 07:03:38,True,False,United States,year,156000.0,NaN,National Technical Information Service,NaN
3,Machine Learning Engineer,Machine Learning Research Scientist,"Pittsburgh, PA",via Ai-Jobs.net,Full-time,False,"Illinois, United States",2023-04-13 16:05:41,False,True,United States,year,140000.0,NaN,Bosch Group,"['pytorch', 'tensorflow']"
4,Data Scientist,"Data Scientist, AWS","Irving, TX",via Snagajob,Full-time and Part-time,False,"Texas, United States",2023-10-15 06:02:51,False,False,United States,hour,NaN,39.795002,"Presidio, Inc.","['python', 'r', 'sql', 'c', 'aws', 'gcp', 'big..."


In [185]:
df.shape

(32672, 16)

### Unique `job_title_short`

In [97]:
job_titles = ( pd.Series(df.job_title_short.unique(), name = 'job_title')
              .sort_values()
              .reset_index(drop = True)
)

In [98]:
job_titles

0             Business Analyst
1               Cloud Engineer
2                 Data Analyst
3                Data Engineer
4               Data Scientist
5    Machine Learning Engineer
6          Senior Data Analyst
7         Senior Data Engineer
8        Senior Data Scientist
9            Software Engineer
Name: job_title, dtype: object

In [99]:
job_titles.to_csv('./csv/job_titles.csv', index = False)

### Unique `company_name`

In [100]:
# df.company_name.nunique()

In [101]:
df.company_name = df.company_name.str.replace('\u200b', '').str.strip()

In [102]:
df.company_name.sort_values()

22939            #twiceasnice Recruiting
22429            #twiceasnice Recruiting
32014            #twiceasnice Recruiting
29447            #twiceasnice Recruiting
29123    (THE VANGUARD GROUP/MALVERN,PA)
                      ...               
22652                         zooplus SE
10615                                ztp
3245                               ЛАНИТ
18882           Технологическая компания
1759                           𝐎𝐩𝐞𝐧𝐬𝐭𝐚𝐟𝐟
Name: company_name, Length: 32672, dtype: object

In [103]:
df.loc[df.company_name.str.startswith('0'), 'company_name']

15279    0nward Select
16632    0nward Select
Name: company_name, dtype: object

`0nward select` has a zero as initial O

In [104]:
df.loc[df.company_name == '0nward Select', 'company_name'] = 'Onward Select'

*Openstaff* has a weird font and won't be recognized as simple string

In [105]:
df.company_name.sort_values().tail(5)

22652                  zooplus SE
10615                         ztp
3245                        ЛАНИТ
18882    Технологическая компания
1759                    𝐎𝐩𝐞𝐧𝐬𝐭𝐚𝐟𝐟
Name: company_name, dtype: object

In [106]:
df.loc[1759, 'company_name'] = 'Openstaff'

Get a text file with all the values for `company_name`

In [107]:
with open('df_companies.txt', 'w') as f:
    for value in df.company_name:
        f.write(f'{value}\n')

Process the file using the script `clean_companies.py` and then read the output file

In [108]:
import clean_companies as clean

In [109]:
# Process the file
clean_companies = clean.process_company_names('df_companies.txt', 'cleaned_companies.txt')

Original number of companies: 32672
Unique normalized companies: 9136


In [110]:
clean_companies

['Cox Communications',
 'Worldgate',
 'National Technical Information Service',
 'Bosch',
 'Presidio',
 'Lmi Consulting',
 'Multiplan',
 'Sports Info',
 'Arsenault',
 'TikTok',
 'Pinterest',
 'Riot Games',
 'Northwest Software',
 'TikTok',
 'TikTok',
 'Forsyth Barnes',
 'Emw Staffing',
 'Volt Technical Resources',
 'Us Army Corps Of Engineers',
 'Capital One',
 'Jobs Near Me',
 'Digirocks',
 'Sofi',
 'Gravity It Resources',
 'Harnham',
 'American Family Mutual Insurance',
 'TikTok',
 'Insight Global',
 'Abbvie',
 'Unitedhealth',
 'Citigroup',
 'Lumen',
 'Eliassen',
 'Motion Recruitment',
 'New York University',
 'Lmi Consulting',
 'Cox Communications',
 'Insight Global',
 'Idc',
 'Office Of General Counsel',
 'Northrop Grumman',
 'TikTok',
 'Kpmg',
 'M Science',
 'Cisco',
 'Federal Reserve System',
 'Kpmg',
 'Cottonwood Financial',
 'Acadia',
 'Aditi Consulting',
 'Cox Communications',
 'Paragon Alpha Hedge Fund Talent Business',
 'Sga',
 'Brooksource',
 'Transtach',
 'Lmi Consulting',

Compare the results

In [111]:
pd.DataFrame({'df' : df.company_name.head(), 'clean' : clean_companies[:5]})

,df,clean
0,Cox Communications,Cox Communications
1,Worldgate LLC,Worldgate
2,National Technical Information Service,National Technical Information Service
3,Bosch Group,Bosch
4,"Presidio, Inc.",Presidio


Finally, reassign the clean company names to the dataframe column

In [112]:
df.company_name = clean_companies

In [113]:
df.company_name.isna().sum()

np.int64(0)

Some `company_name` values are empty (they were Co. Inc., Company... and so on)

In [114]:
df.company_name[df.company_name == '']

1038     
2735     
4318     
24027    
25882    
27635    
29441    
31588    
Name: company_name, dtype: object

In [115]:
df.loc[df.company_name == '', 'company_name'] = 'unknown'

Fix some issues with values that are not estrictly duplicated, but will raise an error as duplicates when loading the data into the database.

In [116]:
df.company_name[df.company_name.str.contains('Mondel')]

16953    Mondelēz
28685    Mondelez
Name: company_name, dtype: object

In [117]:
df.company_name[df.company_name.str.contains('Nestl')]

838                     Nestlé
2240                    Nestlé
4120                    Nestlé
5843                    Nestlé
8896                    Nestlé
9238                    Nestlé
10826                   Nestlé
13061                   Nestle
15780                   Nestlé
16330                   Nestlé
17430                   Nestle
17688                   Nestlé
17784    Nestl Purina Pet Care
17821                   Nestle
21907                   Nestlé
23068                   Nestle
25319                   Nestlé
25890                   Nestle
30503                   Nestlé
Name: company_name, dtype: object

In [118]:
df.loc[df.company_name == 'Nestle', 'company_name'] = 'Nestlé'

In [119]:
df.loc[df.company_name.str.contains('Mondel'), 'company_name'] = 'Mondelez'

Get list of unique companies:

In [120]:
companies = pd.Series(df.company_name.unique(), name = 'company')

In [121]:
companies = companies.sort_values().reset_index(drop=True)

In [122]:
companies.to_csv('./csv/companies.csv', index = False)

### Unique locations

In [123]:
# df.job_country.unique()

In [124]:
countries = pd.Series(df.job_country.unique())

I will add Georgia to the countries table.

In [125]:
countries = pd.concat([countries, pd.Series('Georgia')])
countries = countries.sort_values().reset_index(drop = True)
countries.name = 'country'

In [126]:
# countries.head()

In [127]:
countries.to_csv('./csv/countries.csv', index = False)

#### Unique `job_location`

In [128]:
# df.job_location.unique()

In [129]:
df.job_location = df.job_location.str.strip().fillna('unknown')

There are a couple of duplicates:  
Vilnius, Vilnius City Municipality, Lithuania  
Vilnius, Vilnius city municipality, Lithuania

In [130]:
df.loc[df.job_location.str.contains('Vilnius'), 'job_location'] = 'Vilnius, Vilnius City Municipality, Lithuania'

In [131]:
job_locations = pd.Series(df.job_location.unique(), name = 'location_name')
job_locations = job_locations.sort_values().reset_index(drop = True)

job_locations.head()

0                A Coruña, Spain
1                Aarhus, Denmark
2                Abbott Park, IL
3                      Abell, MD
4    Aberdeen Proving Ground, MD
Name: location_name, dtype: object

In [132]:
job_locations.to_csv('./csv/locations.csv', index = False)

Check if the search_locations are contained either in job_country or job_location

In [133]:
for s in df.search_location.unique():
    if s not in countries.values:
        print(s)

New York, United States
Illinois, United States
Texas, United States
California, United States
Florida, United States


I will set the `search_location` values which contain *United States* (State, United States) to just *United States*

In [134]:
df.loc[df.search_location.str.contains('United States'), 'search_location'] = 'United States'

### Unique `job_schedule_type`

In [135]:
# df.job_schedule_type.unique()

In [136]:
# df.job_schedule_type.head()

In [137]:
df.job_schedule_type = df.job_schedule_type.fillna('NULL')

Convert `job_schedule_type` to list of values so I can explode the dataset later

In [138]:
def convert_schedule_into_list(row):
    if row != 'NULL':
        row = row.replace(' and ', ',').replace(',,', ',').split(',')
        
    # for string with only one schedule, convert to list
    # and return a list
    if isinstance(row, str):
        return [row]
    
    return row

In [139]:
df.job_schedule_type = df.job_schedule_type.apply(convert_schedule_into_list)

In [140]:
all_schedules = []
for i in df.job_schedule_type:
    all_schedules += i
    unique_schedules = list(set(all_schedules))
unique_schedules = [i.strip() for i in unique_schedules if i != 'NULL' ]

schedules = (pd.Series(unique_schedules, name='schedule_type')
             .drop_duplicates()
             .sort_values()
             .reset_index(drop = True)
)

In [141]:
# schedules

In [142]:
schedules.to_csv('./csv/job_schedules.csv', index = False)

### Unique `job_via`

In [143]:
df.job_via = df.job_via.fillna('unknown')

In [144]:
df.job_via = df.job_via.str.replace('via ', '').str.strip()

In [145]:
job_via = pd.Series( sorted(df.job_via.unique(), key=lambda x : -len(x)) )

In [146]:
# job_via

Remove duplicates in the job portals (`job_via`) list.

In [147]:
lowercase_job_via = job_via.str.lower()
job_via_counts = lowercase_job_via.value_counts().sort_values(ascending = False)

In [148]:
job_via_counts[job_via_counts > 1]

informs career center - informs.org    2
linkedin                               2
jobserve                               2
Name: count, dtype: int64

There are 3 duplicate values

In [149]:
job_via[job_via.str.lower().str.contains('inkedin') |
         job_via.str.lower().str.contains('serve')]


61     JobServe - Project Manager Jobs
79        JobServe - Jobs With Walmart
102          JobServe - Developer Jobs
109          JobServe - Bilingual Jobs
113           Maryland Jobs - JobServe
157             LinkedIn Côte D'Ivoire
250                LinkedIn Luxembourg
262                LinkedIn Azerbaijan
369                   LinkedIn Vietnam
378                   LinkedIn Hungary
417                     LinkedIn Malta
452                        IT JobServe
542                           LinkedIn
543                           JobServe
547                           Jobserve
562                           Linkedin
dtype: object

It turned out that many portals can be reduced to just `Jobserve` or `LinkedIn`

In [150]:
job_via[job_via.str.lower().str.contains('inkedin')] = 'LinkedIn'
job_via[job_via.str.lower().str.contains('serve')] = 'JobServe'
job_via[job_via.str.lower().str.contains('informs.org')] = 'Informs Career Center - informs.org'

In [151]:
# [i for i in job_via if ',' in i]

Reduce the job posting sources that contain *Geebo* or *Talentify*

In [152]:
job_via[job_via.str.lower().str.contains('geebo')] = 'geebo.com'
job_via[job_via.str.contains('Talentify')] = 'Talentify'

Remove duplicates

In [153]:
# before / after removing duplicates
job_via.shape, job_via.drop_duplicates().shape

((596,), (321,))

In [154]:
job_via.drop_duplicates(inplace= True)

In [155]:
job_via = job_via.str.replace('Www.', 'www.')

In [156]:
# print(*job_via, sep = '\n')

In [157]:
job_via[job_via == 'InternsVilla | Hub Of Internships -'] = 'InternsVilla | Hub Of Internships'

In [158]:
job_via = pd.Series(job_via, name = 'job_via').sort_values().reset_index(drop = True)

In [159]:
job_via.to_csv('./csv/job_via.csv', index = False)

### Unique `job_skills`

In [160]:
skills = list(df.job_skills.fillna('NULL').unique())

In [161]:
skills = [i.replace('[', '')
          .replace(']', '')
          .replace("'", '')
          for i in skills if i != 'NULL']

In [162]:
skills_expanded = ', '.join(skills).split(', ')
skills = list(set(skills_expanded))

In [163]:
job_skills = pd.Series(skills).sort_values().reset_index(drop = True)
job_skills.name = 'skill'

In [164]:
# job_skills

In [165]:
job_skills.to_csv('./csv/skills.csv', index = False)

Explode the original dataset to generate a row for every skill required in the job posting.

In [166]:
def expand_skills(row):
    if not isinstance(row, float):
        skills_list = (row.replace('[', '')
                    .replace(']', '')
                    .replace("'", '')
        )
        skills_list = skills_list.split(', ')
        return skills_list
    else:
        return row

In [167]:
df.job_skills = df.job_skills.apply(expand_skills)

### Explode the dataset by `job_skills` and `job_schedule_type`

In [168]:
df2 = df.explode('job_schedule_type')
df2 = df2.explode('job_skills')

### Normalize `job_country`

In [169]:
s = pd.Series(countries.index, index = countries.values)
df2.job_country = df2.job_country.map(s)

### Normalize `job_location`

In [170]:
job_locations = job_locations.dropna()

In [171]:
s = pd.Series(job_locations.index, index = job_locations.values)
df2.job_location = df2.job_location.map(s)

### Normalize `job_via`

In [172]:
job_via

0                1840 & Company
1      247 Careers For Freshers
2                    AEC Living
3                    AI Careers
4                   AITechTrend
                 ...           
316               www.hireme.ai
317           www.joblatter.net
318             www.worklis.com
319                          领英
320                      领英(中国)
Name: job_via, Length: 321, dtype: object

In [173]:
s = pd.Series(job_via.index, index = job_via.values)
df2.job_via = df2.job_via.map(s)

In [174]:
df2.head()

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills
0,Senior Data Scientist,Senior Data Scientist Data and Analytics Perfo...,206,312.0,Full-time,False,Sudan,2023-04-24 09:51:15,False,True,94,year,128050.0,NaN,Cox Communications,sql
0,Senior Data Scientist,Senior Data Scientist Data and Analytics Perfo...,206,312.0,Full-time,False,Sudan,2023-04-24 09:51:15,False,True,94,year,128050.0,NaN,Cox Communications,python
0,Senior Data Scientist,Senior Data Scientist Data and Analytics Perfo...,206,312.0,Full-time,False,Sudan,2023-04-24 09:51:15,False,True,94,year,128050.0,NaN,Cox Communications,aws
0,Senior Data Scientist,Senior Data Scientist Data and Analytics Perfo...,206,312.0,Full-time,False,Sudan,2023-04-24 09:51:15,False,True,94,year,128050.0,NaN,Cox Communications,pyspark
0,Senior Data Scientist,Senior Data Scientist Data and Analytics Perfo...,206,312.0,Full-time,False,Sudan,2023-04-24 09:51:15,False,True,94,year,128050.0,NaN,Cox Communications,tableau


### Normalize `job_schedule_type`

In [175]:
s = pd.Series(schedules.index, index = schedules.values)
df2.job_schedule_type = df2.job_schedule_type.map(s)

### Normalize `search_location`

In [176]:
s = pd.Series(countries.index, index = countries.values)
df2.search_location = df2.search_location.map(s)

### Normalize `job_company`

In [177]:
s = pd.Series(companies.index, index = companies.values)
df2.company_name = df2.company_name.map(s)

### Normalize `job_skills`

In [178]:
s = pd.Series(job_skills.index, index = job_skills.values)
df2.job_skills = df2.job_skills.map(s)

### Normalize `job_title_short`

In [179]:
s = pd.Series(job_titles.index, index = job_titles.values)
df2.job_title_short = df2.job_title_short.map(s)

In [180]:
df2.sample(4)

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills
20564,4,Data Scientist - Now Hiring,1888,240.0,1.0,False,106,2023-12-10 07:05:51,False,True,106,hour,NaN,43.709999,4160,15.0
20587,2,Data Analyst - Top Secret Required - Now Hiring,2256,240.0,3.0,False,106,2023-09-21 06:01:43,False,False,106,hour,NaN,18.795000,4644,149.0
1594,7,Senior Data Engineer & Innovation - FULL-TIME ...,85,181.0,1.0,True,106,2023-01-19 19:49:44,False,False,106,year,95000.0,NaN,7286,14.0
17595,7,Senior Data Engineer,1767,14.0,1.0,False,36,2023-05-16 09:29:37,False,False,36,year,147500.0,NaN,8163,48.0


In [181]:
df2.dtypes

job_title_short                   int64
job_title                        object
job_location                      int64
job_via                         float64
job_schedule_type               float64
job_work_from_home                 bool
search_location                   int64
job_posted_date          datetime64[ns]
job_no_degree_mention              bool
job_health_insurance               bool
job_country                       int64
salary_rate                      object
salary_year_avg                 float64
salary_hour_avg                 float64
company_name                      int64
job_skills                      float64
dtype: object

In [182]:
df2 = df2.sort_values('job_posted_date').reset_index(drop = True)

In [183]:
df2.rename(columns = {
    'job_title_short' : 'job_title_id',
    'job_title' : 'job_title_full',
    'job_location' : 'job_location_id',
    'job_via' : 'job_via_id',
    'job_schedule_type' : 'schedule_id',
    'search_location' : 'search_location_id',
    'job_location' : 'job_location_id',
    'job_work_from_home' : 'work_from_home',
    'job_no_degree_mention' : 'no_degree_mention',
    'job_health_insurance' : 'health_insurance',
    'job_country' : 'job_country_id',
    'company_name' : 'company_id',
    'job_skills' : 'skill_id'
}, inplace = True)

In [184]:
df2.to_csv('./csv/job_postings.csv', index = False)